# Fine-tuning RoBERTa

In [ ]:
import glob
import numpy as np
import os
import pandas as pd
import psutil
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_scheduler

In [ ]:
if torch.cuda.is_available(): device = torch.device("cuda")
elif torch.backends.mps.is_available(): device = torch.device("mps")
else: device = torch.device("cpu")

In [ ]:
os.makedirs("output/models/", exist_ok=True)
os.makedirs("output/preds/", exist_ok=True)

In [ ]:
train = pd.read_csv("data/processed/train.csv")
test = pd.read_csv("data/processed/test.csv")

In [ ]:
# train = train.sample(frac=0.005).reset_index(drop=True)
# test = test.sample(frac=0.005).reset_index(drop=True)

# 1. Build the model

Firstly, let's load the **base model** we intend to fine-tune. RoBERTa is initialized with an untrained classifier head consisting of two dense (= linear) layers:
- The first projects the output from the base model to a hidden space of the same dimensionality (1024 for RoBERTa-large).
- The second maps the hidden representation to the label space, reducing the dimensionality accordingly (to 2 for a binary classifier).

This classifier head is not pretrained, hence the warning below. It can either be fine-tuned directly or replaced with a custom classifier block.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('roberta-large')

Fine-tuning is usually restricted to the final layers of the model responsible for the desired task. This approach speeds up training while preserving the knowledge acquired by the model during pretraining. Since we plan to implement a custom classifier block, I simply freeze all parameters of the base model (they are still trainable by default).

In [ ]:
# Freeze pretrained layers 
print(f"Number of parameters in base RoBERTa: {sum(p.numel() for p in model.parameters()) / 1e6:.2f} M")

for param in model.parameters(): 
    param.requires_grad = False

print(f"Number of trainable parameters after freezing: {sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.2f}M")

There are two key decisions for the classifier head: which layers to include, and which input to use.

Regarding the **layers**, a typical approach is to insert an activation layer and a dropout layer between the two projection layers of the initial classification head. The activation layer introduces non-linearity, while the dropout layer helps prevent overfitting. I also add a normalization layer and another dropout layer after the activation layer, as well as an intermediate dense layer to avoid going at once from dimension 1024 to 2. A simpler classifier head can be obtained by retaining only the last 2 dense layers. A final Softmax layer is necessary to obtain probabilities, and it is typically placed in the `predict` function (instead of `forward`) to simplify the computation of the loss.

Regarding the **input**:
- The simplest approach is to focus on the `[CLS]` token, which marks the beginning of the input sequence and serves as an embedding for its entirety: `x = outputs.last_hidden_state[:, 0, :]`
- Another approach is to pool all tokens from the input sequence, either by taking their mean or maximum in each dimension: `x = torch.mean(outputs.last_hidden_state, dim=1)` or  `x = torch.max(outputs.last_hidden_state, dim=1).values`.

In [ ]:
# Add layers for classification to finetune
class RoBERTa_architecture(nn.Module): 
    def __init__(self, roberta_model): 
        super(RoBERTa_architecture, self).__init__() 
        self.roberta = roberta_model
        # Dense layer 1 => input layer
        self.in_proj = nn.Linear(1024, 1024)  # Disable for simplex model
        # Dropout layer
        self.dropout = nn.Dropout(0.1)  # Disable for simplex model
        # GELU
        self.gelu = nn.GELU()  # Disable for simplex model
        # Layer nom
        self.layer_norm = nn.LayerNorm(1024, eps=1e-5)  # Disable for simplex model
        # Dense layer 2 => intermediate layer
        self.inter_proj = nn.Linear(1024, 512)
        # Dense layer 3 => output layer
        self.out_proj = nn.Linear(512, 2)

    def forward(self, sent_id, mask): 
        # Pass inputs to the base model
        outputs = self.roberta(sent_id, attention_mask=mask, return_dict=True)
        # Get CLS token
        x = outputs.last_hidden_state[:, 0, :]
        # Apply new layers
        x = self.in_proj(x)  # Disable for simplex model
        x = self.dropout(x)  # Disable for simplex model
        x = self.gelu(x)  # Disable for simplex model
        x = self.layer_norm(x)  # Disable for simplex model
        x = self.inter_proj(x)
        x = self.dropout(x)  # Disable for simplex model
        x = self.out_proj(x)
        return x
    
    def predict(self, sent_id, mask):
        self.eval()
        # Disable gradient calculation for inference
        with torch.no_grad():
            logits = self.forward(sent_id, mask)
        # Apply Softmax
        probs = F.softmax(logits, dim=1)
        # Retain highest probability
        preds = torch.argmax(probs, dim=1)
        return preds

In [ ]:
# Create the model
roberta_classifier = RoBERTa_architecture(model.roberta)
roberta_classifier = roberta_classifier.to(device)

# Map the labels
id2label = {0: "negative", 1: "positive"}
label2id = {"negative": 0, "positive": 1}

roberta_classifier.id2label = id2label
roberta_classifier.label2id = label2id

In [ ]:
# Chech which layers are available for training
for name, param in roberta_classifier.named_parameters(): 
    if param.requires_grad:
        print(name)

As expected, only 4 of the new layers are trainable:
- The first dense layer, making for $1024*1024+1024=1,049,600$ trainable parameters
- The normalization layer, making for $1024*2=2,048$ trainable parameters
- The secund dense layer, making for $1024*512+512=524,800$ trainable parameters
- The final dense layer, making for $512*2+2=1,026$ trainable parameters

The total number of trainable parameters should thus be $1,577,474$ (or $525,826$ for the simpler model).

In [ ]:
print(f"Number of parameters to fine-tune: {sum(p.numel() for p in roberta_classifier.parameters() if p.requires_grad)}")

# 2. Preprocessing
First, we need a tokenizer. RoBERTa uses Byte-Pair Encoding (BPE) which operates at the subword level with an agglomerative approach. It merges the most frequent pairs iteratively until the desired vocabulary size is reached, which is 50,265 for RoBERTa. 

RoBERTa has a maximum context length of 512 tokens. Sequences longer than this must be truncated before being passed to the model. We opt for left-side truncation to preserve summaries that sometimes appear at the end of reviews. An alternative approach would be to split long sequences into smaller chunks, process them sequentially, and average the results. However, this would be more complex to implement and may not yield reliable results.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('roberta-large', truncation_side = 'left')

RoBERTa requires all input sequences to be of the same length. Longer sequences are truncated, as discussed above, while shorter sequences are padded by adding empty tokens like `[PAD]` at the end. A mask is also generated to help the model identify the padded tokens. Padding can be applied either to the maximum context length of the model or to the longest sequence in the batch, which is the option used here. RoBERTa does not use `token_type_ids`, which are typically employed to distinguish between different parts of the input, such as prompts and answers. As a result, they are disabled here. Lastly, we pass the `review_id` variable to the `dataloader` so that we can match the predictions with the original dataset once the model is trained.

`batch_size` defines how many sequences are processed in parallel during a single batch. Larger batch sizes can improve computational efficiency, provided there is enough available memory. They offer more stable gradient estimates by using larger amounts of data, but they result in less frequent updates to the model. As a consequence, larger batch sizes may require more epochs to converge, as well as a lower learning rate to reduce the risk of overfitting. Here, `batch_size` is set to 250, meaning that 1% of the dataset is processed in each batch. This setup consumes up to 25 Go of memory. There is no downside to using a higher `batch_size` for inference, so we set it to 400 to fully utilize the 64 GB of available memory on our machine.

In [ ]:
class DataPreprocessor:
    def __init__(self, tokenizer, label2id):
        self.tokenizer = tokenizer
        self.train_batch_size = 250
        self.test_batch_size = 400
        self.label2id = label2id

    def prepare(self, data, is_train=True):
        tokens = self.tokenizer(
            data['text'].tolist(),
            padding = 'longest',
            truncation = True,
            return_token_type_ids = False,
            return_tensors = 'pt')
        # Tokenize and convert to tensor
        input_ids = tokens['input_ids'].clone().detach()
        attention_mask = tokens['attention_mask'].clone().detach()
        # Get sequence lengths
        seq_lengths = [len(seq) for seq in input_ids]
        # Convert labels
        numeric_labels = [self.label2id[label] for label in data['sentiment']]
        labels = torch.tensor(numeric_labels, dtype = torch.long)
        # Extract review IDs
        ids = torch.tensor(data['review_id'].tolist(), dtype = torch.long)
        # Choose the appropriate batch size
        batch_size = self.train_batch_size if is_train else self.test_batch_size
        # Create dataset and dataloader
        dataset = TensorDataset(input_ids, attention_mask, labels, ids)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=is_train)

        return dataloader, seq_lengths

In [ ]:
time0 = time.time()

preprocessor = DataPreprocessor(tokenizer, label2id)
train_dataloader, train_seq_lengths = preprocessor.prepare(train, is_train=True)
test_dataloader, test_seq_lengths = preprocessor.prepare(test, is_train=False)

time1 = time.time()
print(f"Tokenization duration: {time1-time0:.2f} seconds")

In [ ]:
print(f"Average length of sequences in train: {sum(train_seq_lengths) / len(train_seq_lengths):.0f}")
print(f"Average length of sequences in test: {sum(test_seq_lengths) / len(test_seq_lengths):.0f}")

After tokenization and dataloading, the average length of sequences in 512. This indicates that there is at least one sequences longer than 512 tokens in each batch, so that `padding = 'longest'` has the same effect as `padding = 'max_length'`. With a `batch_size` of 250, there are exactly $250×512=128,000$ tokens per (training) batch.

# 3. Training

We define functions for **training** and **testing**. Since we plan to evaluate the model on the entire test dataset after each epoch, we store the logits returned at that time. This allows us to retrieve predictions directly, avoiding the need to recompute them. Additionally, we save the true labels and review IDs to link the predictions with the original dataset for analysis.

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
batch_train_losses = []
batch_train_accuracy = []

def model_train(batch):
    b_input_ids, b_attention_mask, b_labels, _ = [t.to(device) for t in batch]
    # Reset gradients before backpropagation
    roberta_classifier.zero_grad()
    # Perform a forward pass to calculate outputs
    logits = roberta_classifier(b_input_ids, b_attention_mask)
    # Calculate the loss
    loss = loss_fn(logits, b_labels)
    batch_train_losses.append(loss.item())
    # Calculate accuracy
    preds = torch.argmax(logits, dim=1)
    batch_train_accuracy.append((preds == b_labels).sum().item() / b_labels.size(0))
    # Backpropagate the loss
    loss.backward()
    # Update model parameters
    optimizer.step()
    
    return loss

In [ ]:
batch_test_losses = []
batch_test_accuracy = []

def model_eval(batch):
    b_input_ids, b_attention_mask, b_labels, b_ids = [t.to(device) for t in batch]
    # Forward pass
    logits = roberta_classifier(b_input_ids, b_attention_mask)
    # Store results 
    all_logits.append(logits.detach().cpu())
    all_labels.append(b_labels.detach().cpu())
    all_ids.append(b_ids.detach().cpu())
    # Calculate loss
    loss = loss_fn(logits, b_labels)
    batch_test_losses.append(loss.item())
    # Calculate accuracy
    preds = torch.argmax(logits, dim=1)
    batch_test_accuracy.append((preds == b_labels).sum().item() / b_labels.size(0))
    
    return loss

We define a **training loop** with the ability to resume after interruption.

- The training dataset is processed `num_epochs` times. Typical values for fine-tuning range between 3 to 12. We set it to 12 and will compute the test error after every batch to detect when the model starts overfitting.

- For the **optimizer**, we set a relatively low learning rate, as is commonly recommended for fine-tuning RoBERTa models. The `weight_decay` parameter is a regularization term that adds a penalty for large weights during optimization, helping to mitigate overfitting.

- We use a **scheduler** to further reduce the learning rate as the model converges. The `num_warmup_steps` is another regularization parameter, which gradually reduces the learning rate during the first batches of each epoch to prevent instability.

In [ ]:
num_epochs = 22
num_training_steps = num_epochs * len(train_dataloader)
print(f"Number of training steps: {num_training_steps}")

# Optimizer
optimizer = torch.optim.AdamW(
    roberta_classifier.parameters(),
    lr = 1e-4,
    weight_decay = 0.03,
    eps = 1e-6)

# Scheduler
lr_scheduler = get_scheduler(
    "cosine",
    optimizer = optimizer,
    num_warmup_steps = 0.1 * num_training_steps,
    num_training_steps = num_training_steps)

# Avoid issues with multithreading
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# Resume training
def get_latest_checkpoint(model_dir):
    # Verify if checkpoints are present
    checkpoint_files = glob.glob(os.path.join(model_dir, "epoch_*.pth"))
    if not checkpoint_files:
        return None, 0
    # Extract epoch numbers from filenames
    epochs = [int(os.path.basename(f).split("_")[1].split(".")[0]) for f in checkpoint_files]
    last_epoch = max(epochs)
    return os.path.join(model_dir, f"epoch_{last_epoch}.pth"), last_epoch

latest_checkpoint, last_epoch = get_latest_checkpoint("output/models")

if latest_checkpoint:
    if last_epoch >= num_epochs:
        print(f"Training already completed up to epoch {last_epoch}.")
    else:
        checkpoint = torch.load(latest_checkpoint, weights_only=False)
        roberta_classifier.load_state_dict(checkpoint['model'], strict=False)  # Load saved parameters only
        optimizer.load_state_dict(checkpoint['optimizer'])
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
        batch_train_losses = checkpoint['batch_train_losses']
        batch_test_losses = checkpoint['batch_test_losses']
        epoch_train_losses = checkpoint['epoch_train_losses']
        epoch_test_losses = checkpoint['epoch_test_losses']
        batch_train_accuracy = checkpoint['batch_train_accuracy']
        batch_test_accuracy = checkpoint['batch_test_accuracy']
        epoch_train_accuracy = checkpoint['epoch_train_accuracy']
        accuracies = checkpoint['accuracies']
        classif_errors = checkpoint['classif_errors']
        batch_times_train = checkpoint['batch_times_train']
        batch_times_test = checkpoint['batch_times_test']
        epoch_times = checkpoint['epoch_times']
        print(f"Resuming from epoch {last_epoch}")

else:
    epoch_train_losses = []
    epoch_test_losses = []
    epoch_train_accuracy = []
    accuracies = []
    classif_errors = []
    batch_times_train = []
    batch_times_test = []
    epoch_times = []
    print(f"No checkpoint found, starting over...")

In [ ]:
# Memory tracking
if device == 'mps':
    torch.mps.empty_cache()
baseline_memory = psutil.virtual_memory().available
max_memory_test = 0
max_memory_train = 0

In [ ]:
epoch_progress = tqdm(range(last_epoch, num_epochs),
                      desc = "Epochs",
                      position = 0,
                      unit = "epoch")

for epoch in epoch_progress:
    epoch_st = time.time()
    
    # =============================
    #           Training
    # =============================
    # Set up the train progress bar
    train_progress = tqdm(total = len(train_dataloader), 
                          desc = f"Train", 
                          position = 1, 
                          leave = False,
                          unit = "batch")
    
    roberta_classifier.train()
    for batch in train_dataloader:
        batch_st = time.time()
        loss = model_train(batch)
        # Update progress bar with current loss
        train_progress.set_postfix(loss=f"{loss.item():.4f}")
        train_progress.update(1)
        # Track memory use and computation time
        max_memory_train = max(max_memory_train, psutil.virtual_memory().available)
        batch_times_train.append(time.time() - batch_st)
    train_progress.close()
    
    # Calculate average train loss over the epoch
    avg_train_loss = np.mean(batch_train_losses[-len(train_dataloader):])
    epoch_train_losses.append(avg_train_loss)
    # Calculate average train accuracy over the epoch
    avg_train_accuracy = np.mean(batch_train_accuracy[-len(train_dataloader):])
    epoch_train_accuracy.append(avg_train_accuracy)
    # Update the learning rate
    lr_scheduler.step()

    
    # =============================
    #            Testing
    # =============================
    # Set up the test progress bar
    test_progress = tqdm(total = len(test_dataloader), 
                         desc = f"Test", 
                         position = 2, 
                         leave = False,
                         unit = "batch")
    
    all_logits = []
    all_labels = []
    all_ids = []
    
    roberta_classifier.eval()
    for batch in test_dataloader:
        batch_st = time.time()
        loss = model_eval(batch)
        # Update progress bar with current loss
        test_progress.set_postfix(loss=f"{loss.item():.4f}")
        test_progress.update(1)
        # Track memory use and computation time
        max_memory_test = max(max_memory_test, psutil.virtual_memory().available)
        batch_times_test.append(time.time() - batch_st)
    test_progress.close()
    
    # Calculate average test loss over the epoch
    avg_test_loss = np.mean(batch_test_losses[-len(test_dataloader):])
    epoch_test_losses.append(avg_test_loss)
    # Calculate classification error
    all_logits = torch.cat(all_logits, dim=0)
    probs = F.softmax(all_logits, dim=1).detach()
    prob_class_0 = probs[:, 0]
    prob_class_1 = probs[:, 1]
    classif_error = (1 - torch.max(prob_class_0, prob_class_1)).mean().item()
    classif_errors.append(classif_error)
    # Calculate accuracy
    preds_array = torch.argmax(probs, dim=1).cpu().numpy()
    labels_array = torch.cat(all_labels, dim=0).cpu().numpy()
    accuracy = accuracy_score(preds_array, labels_array)
    accuracies.append(accuracy)
    
    
    # =============================
    #             Saving
    # =============================
    # Update epoch progress bar with average losses
    epoch_progress.set_postfix(train_loss=f"{avg_train_loss:.4f}", test_loss=f"{avg_test_loss:.4f}", accuracy=f"{accuracy:.4f}")
    # Save predictions
    probs_array = probs.cpu().numpy()
    results = pd.DataFrame(probs_array, columns=[f"prob_class_{i}" for i in range(probs_array.shape[1])])
    results['true_label'] = [id2label[label] for label in labels_array]
    results['review_id'] = torch.cat(all_ids, dim=0).detach().cpu().numpy()
    results.to_csv(f"output/preds/epoch_{epoch+1}.csv", index=False)
    # Track computation time
    epoch_times.append(time.time() - epoch_st)
    # Save model checkpoint
    model_save_path = os.path.join("output/models", f"epoch_{epoch+1}.pth")
    checkpoint = {
        'epoch': epoch,
        'model': {k: v for k, v in roberta_classifier.named_parameters() if v.requires_grad},
        'optimizer': optimizer.state_dict(),
        'lr_scheduler': lr_scheduler.state_dict(),
        'batch_train_losses': batch_train_losses,
        'batch_test_losses': batch_test_losses,
        'epoch_train_losses': epoch_train_losses,
        'epoch_test_losses': epoch_test_losses,
        'batch_train_accuracy': batch_train_accuracy,
        'batch_test_accuracy': batch_test_accuracy,
        'epoch_train_accuracy': epoch_train_accuracy,
        'accuracies': accuracies,
        'classif_errors': classif_errors,
        'batch_times_train': batch_times_train,
        'batch_times_test': batch_times_test,
        'epoch_times': epoch_times}
    torch.save(checkpoint, model_save_path)
    # Clean memory
    if device == 'mps':
        torch.mps.empty_cache()

epoch_progress.close()

In [ ]:
print(f"Max memory used for training is {(baseline_memory - max_memory_train) / 1e9:.2f} GB")
print(f"Max memory used for evaluating is {(baseline_memory - max_memory_test) / 1e9:.2f} GB")

In [ ]:
print(f"Average computation time for training batches: {np.mean(batch_times_train):.2f} seconds")
print(f"Average computation time for validation batches: {np.mean(batch_times_test):.2f} seconds")
print(f"Average computation time per epoch: {np.mean(epoch_times):.2f} seconds")
print(f"Total computation time: {np.sum(epoch_times):.2f} seconds")

# 3. Results

In [ ]:
# Select the model with the highest accuracy
latest_checkpoint, last_epoch = get_latest_checkpoint("output/models")
checkpoint = torch.load(latest_checkpoint, weights_only = False)     
accuracies = checkpoint['accuracies']
classif_errors = checkpoint['classif_errors']
best_epoch = accuracies.index(max(accuracies)) + 1
print(f"Best epoch: {best_epoch}")
print(f"Accuracy: {accuracies[best_epoch-1]}")

# Load its predictions
results = pd.read_csv(f"output/preds/epoch_{best_epoch}.csv")
results = pd.merge(test, results, on = 'review_id')

In [ ]:
# Check for consistency
print(f"Do the true labels returned by the model match the original sentiments?")
print(f"Yes!" if (results['sentiment'] == results['true_label']).all() else f"No :'(")

In [ ]:
# Get predicted sentiments and save
results['RoBERTa_ft'] = np.where(results['prob_class_1'] >= 0.5, 'positive', 'negative')
results[['review_id', 'RoBERTa_ft']].to_csv("output/RoBERTa_ft.csv", index=False)